# Linear Regression Model Training
### Read in features and labels
### Do some multivariate linear regression
### See how good the fits are
### Make some plots of the final WhereSIOUS scores & print some stats

In [ ]:
# imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
from scipy.stats.stats import pearsonr
from sklearn import cross_validation

%matplotlib inline

In [ ]:
# define extra bit for filenames for codes w/ # businesses >3000 (equivalently >2100)
extras = ['54','81','45','62','56','44','72','53']
# add codes w/ # businesses <2100 & >500
#extras = [extras, ]'42','23','48','61','71','52','51','33']
# add codes w/ # businesses <500 & >50
#extras = [extras, '32','31','49','55','11']
# add codes w/ # businesses <50
#extras = [extras, '22','92','21']

# pick which code to look at
i = 0
extra = '_'+extras[i]

In [ ]:
# save min/max of business age
bus_ages = pd.read_csv('data_final/sd_active_businesses_cleaned.csv')
# use only current business code
bus_ages = bus_ages[bus_ages.naics_code_simple==float(extras[i])]
# drop tract 9902 (mostly SD Harbor, not much land) is always best tract, w/ only 3 quite old businesses (10-30 yr)
bus_ages = bus_ages[bus_ages.census_tract != 9902]

# get median ages in each tract
med_ages = bus_ages.groupby('census_tract').median().days_since_creation
bus_age_min = min(med_ages)/365.
bus_age_max = max(med_ages)/365.
print bus_age_min
print bus_age_max

In [ ]:
# read in features and labels/examples (see build_features_labels2.ipynb for more info)
features = pd.read_csv('data_final/features'+extra+'.csv')
print features.shape
print features.columns
features[-100:]

## Fit w/ all input features (using statsmodels)

In [ ]:
# fit the data
#   + to use feature
#   - to remove feature
#   : to multiply 2 features
#   * to multiply 2 features *and* use both individually
#   C(<feature>) to make <feature> into dummy/categorical feature
formula_str = 'bus_age ~ n_business - n_businesses_11 - n_businesses_21 - n_businesses_22 ' \
                '- n_businesses_23 - n_businesses_31 - n_businesses_32 - n_businesses_33 - n_businesses_42 ' \
                '- n_businesses_44 - n_businesses_45 - n_businesses_48 - n_businesses_49 - n_businesses_51 ' \
                '- n_businesses_52 - n_businesses_53 - n_businesses_54 - n_businesses_55 - n_businesses_56 ' \
                '- n_businesses_61 - n_businesses_62 - n_businesses_71 - n_businesses_72 - n_businesses_81 ' \
                '- n_businesses_92 + population - population_percent_male + population_percent_female + age_med ' \
                '- age_med_male - age_med_female - minor_percent + adult_percent + homes - homes_percent_mortgage '\
                '- homes_percent_clear - homes_percent_rent + employment_percent + income_med - income_med_owner '\
                '- income_med_renter + cost_med - cost_med_owner - cost_med_renter + walk_score + transit_score '\
                '+ bike_score - total_amount'
#                '+ n_business:population + n_businesses_11:population + n_businesses_21:population + n_businesses_22:population ' \
#                '+ n_businesses_23:population + n_businesses_31:population + n_businesses_32:population + n_businesses_33:population + n_businesses_42:population ' \
#                '+ n_businesses_44:population + n_businesses_45:population + n_businesses_48:population + n_businesses_49:population + n_businesses_51:population ' \
#                '+ n_businesses_52:population + n_businesses_53:population + n_businesses_54:population + n_businesses_55:population + n_businesses_56:population ' \
#                '+ n_businesses_61:population + n_businesses_62:population + n_businesses_71:population + n_businesses_72:population + n_businesses_81:population ' \
#                '+ n_businesses_92:population'
#                '+ n_business:homes + n_businesses_11:homes + n_businesses_21:homes + n_businesses_22:homes ' \
#                '+ n_businesses_23:homes + n_businesses_31:homes + n_businesses_32:homes + n_businesses_33:homes + n_businesses_42:homes ' \
#                '+ n_businesses_44:homes + n_businesses_45:homes + n_businesses_48:homes + n_businesses_49:homes + n_businesses_51:homes ' \
#                '+ n_businesses_52:homes + n_businesses_53:homes + n_businesses_54:homes + n_businesses_55:homes + n_businesses_56:homes ' \
#                '+ n_businesses_61:homes + n_businesses_62:homes + n_businesses_71:homes + n_businesses_72:homes + n_businesses_81:homes ' \
#                '+ n_businesses_92:homes'
#                '+ n_business:income_med + n_businesses_11:income_med + n_businesses_21:income_med + n_businesses_22:income_med ' \
#                '+ n_businesses_23:income_med + n_businesses_31:income_med + n_businesses_32:income_med + n_businesses_33:income_med + n_businesses_42:income_med ' \
#                '+ n_businesses_44:income_med + n_businesses_45:income_med + n_businesses_48:income_med + n_businesses_49:income_med + n_businesses_51:income_med ' \
#                '+ n_businesses_52:income_med + n_businesses_53:income_med + n_businesses_54:income_med + n_businesses_55:income_med + n_businesses_56:income_med ' \
#                '+ n_businesses_61:income_med + n_businesses_62:income_med + n_businesses_71:income_med + n_businesses_72:income_med + n_businesses_81:income_med ' \
#                '+ n_businesses_92:income_med'
#                '+ n_business:cost_med + n_businesses_11:cost_med + n_businesses_21:cost_med + n_businesses_22:cost_med ' \
#                '+ n_businesses_23:cost_med + n_businesses_31:cost_med + n_businesses_32:cost_med + n_businesses_33:cost_med + n_businesses_42:cost_med ' \
#                '+ n_businesses_44:cost_med + n_businesses_45:cost_med + n_businesses_48:cost_med + n_businesses_49:cost_med + n_businesses_51:cost_med ' \
#                '+ n_businesses_52:cost_med + n_businesses_53:cost_med + n_businesses_54:cost_med + n_businesses_55:cost_med + n_businesses_56:cost_med ' \
#                '+ n_businesses_61:cost_med + n_businesses_62:cost_med + n_businesses_71:cost_med + n_businesses_72:cost_med + n_businesses_81:cost_med ' \
#                '+ n_businesses_92:cost_med'
#                '+ n_business:walk_score + n_businesses_11:walk_score + n_businesses_21:walk_score + n_businesses_22:walk_score ' \
#                '+ n_businesses_23:walk_score + n_businesses_31:walk_score + n_businesses_32:walk_score + n_businesses_33:walk_score + n_businesses_42:walk_score ' \
#                '+ n_businesses_44:walk_score + n_businesses_45:walk_score + n_businesses_48:walk_score + n_businesses_49:walk_score + n_businesses_51:walk_score ' \
#                '+ n_businesses_52:walk_score + n_businesses_53:walk_score + n_businesses_54:walk_score + n_businesses_55:walk_score + n_businesses_56:walk_score ' \
#                '+ n_businesses_61:walk_score + n_businesses_62:walk_score + n_businesses_71:walk_score + n_businesses_72:walk_score + n_businesses_81:walk_score ' \
#                '+ n_businesses_92:walk_score'
model = smf.ols(formula=formula_str, data=features).fit()

print len(model.params)
# print the coefficients
model.params

In [ ]:
# RMSE of model
pred = model.predict(features).values * (bus_age_max-bus_age_min) + bus_age_min
actual = features.bus_age.values * (bus_age_max-bus_age_min) + bus_age_min
RMSE = np.sqrt(np.sum((pred-actual)**2)/len(features))
print RMSE

# print a summary of the model
model.summary()

## Leave One Out Cross-Validation

In [ ]:
# define LOO for looping purposes
loo = cross_validation.LeaveOneOut(len(features))

# initialize resulting RMSE
RMSE_loo = []

# go through each example as the hold-out
for train_index, test_index in loo:
    # train model
    model_loo = smf.ols(formula=formula_str, data=features.loc[train_index]).fit()
    # get result from hold-out
    prediction = model_loo.predict(features.loc[test_index])
    # rescale to actual years
    pred = prediction.values * (bus_age_max-bus_age_min) + bus_age_min
    actual = features.loc[test_index].bus_age.values * (bus_age_max-bus_age_min) + bus_age_min
    # save current result 
    RMSE_loo.append((abs(pred-actual))[0])

# take mean of all results
RMSE_loo_arr = np.array(RMSE_loo)
print "LOOCV (median): " + str(np.median(RMSE_loo_arr))
loocv = RMSE_loo_arr.mean()
print "LOOCV = " + str(loocv)

## Predict new ages for all census tracts

In [ ]:
# read in features and labels/examples (see build_features_labels2.ipynb for more info)
features_all = pd.read_csv('data_final/features.csv')
features_all = features_all.drop('n_businesses_11', 1)
features_all = features_all.drop('n_businesses_21', 1)
features_all = features_all.drop('n_businesses_22', 1)
features_all = features_all.drop('n_businesses_23', 1)
features_all = features_all.drop('n_businesses_31', 1)
features_all = features_all.drop('n_businesses_32', 1)
features_all = features_all.drop('n_businesses_33', 1)
features_all = features_all.drop('n_businesses_42', 1)
features_all = features_all.drop('n_businesses_44', 1)
features_all = features_all.drop('n_businesses_45', 1)
features_all = features_all.drop('n_businesses_48', 1)
features_all = features_all.drop('n_businesses_49', 1)
features_all = features_all.drop('n_businesses_51', 1)
features_all = features_all.drop('n_businesses_52', 1)
features_all = features_all.drop('n_businesses_53', 1)
features_all = features_all.drop('n_businesses_54', 1)
features_all = features_all.drop('n_businesses_55', 1)
features_all = features_all.drop('n_businesses_56', 1)
features_all = features_all.drop('n_businesses_61', 1)
features_all = features_all.drop('n_businesses_62', 1)
features_all = features_all.drop('n_businesses_71', 1)
features_all = features_all.drop('n_businesses_72', 1)
features_all = features_all.drop('n_businesses_81', 1)
features_all = features_all.drop('n_businesses_92', 1)
print features_all.shape
print features_all.columns
features_all[-90:-20]

In [ ]:
# make predictions for this code for all tracts
bus_age_pred_final = model.predict(features_all)
tracts_final = features_all.census_tract.values

# recalc model output ages
bus_age_pred = model.predict(features)
#print bus_age_pred[:20]
#print bus_age_pred_final[:20]

In [ ]:
# check stuff that will be saved
print tracts_final.shape
#print tracts_final[:400]
print bus_age_pred_final.shape
#bus_age_pred_final[:400]

## Plot ages, model output ages, and model output ages for adding a business to each tract/code combo

In [ ]:
# make histogram of scaled ages input and output and final output (for each business/tract combo)
plt.figure(figsize=(13, 10))
plt.hist([bus_age_pred_final,bus_age_pred,features.bus_age], bins=30, histtype='step', align='mid', linewidth=2, \
         label=['Predicted Ages for all Tracts','Model Predicted Ages','Actual Ages'])
plt.legend(loc=0,fontsize=20)
plt.grid()
plt.tick_params(axis='both', which='major', labelsize=25)
plt.xlabel('Scaled Age',fontsize=35)
plt.ylabel('# of Census Tracts',fontsize=35)

# unscale actual busisness ages
t1 = features.bus_age * (bus_age_max-bus_age_min) + bus_age_min
# unscale predicted busisness ages
t2 = bus_age_pred * (bus_age_max-bus_age_min) + bus_age_min

plt.figure(figsize=(13, 10))
plt.hist([t2,t1], bins=25, histtype='step', align='mid', linewidth=2, \
         label=['Model Predicted Ages','Actual Ages'])
plt.legend(loc=0,fontsize=20)
plt.grid()
plt.tick_params(axis='both', which='major', labelsize=25)
plt.xlabel('Median Age of Business in Years',fontsize=35)
plt.ylabel('# of Neighborhoods',fontsize=35)

# print some stats about the model output
print features.bus_age.shape
print bus_age_pred.shape
print np.mean(bus_age_pred)
print np.median(bus_age_pred)
print np.std(bus_age_pred)
print

# print some stats about the model output for each tract/code combo
print bus_age_pred_final.shape
print np.mean(bus_age_pred_final)
print np.median(bus_age_pred_final)
print np.std(bus_age_pred_final)

## Save newly made lists of tract/code combos and scores and diff to dataframes

In [ ]:
# scale ages to 0 to 100 to get WhereSIOUS score
print min(bus_age_pred_final)
print max(bus_age_pred_final)
score = 100 * (bus_age_pred_final-min(bus_age_pred_final)) / (max(bus_age_pred_final)-min(bus_age_pred_final))
print min(score)
print max(score)

# scale scores again with a logistic function to spread them out more evenly
score = score/(1+3/np.std(score)*np.exp(-2/np.std(score)*(score-np.mean(score))))
print min(score)
print max(score)

# save lists of tracts and final predicted ages as a dataframe
df_ages = pd.DataFrame({'census_tract': tracts_final, 'score': score})
print df_ages.shape
#print df_ages[:538]

In [ ]:
# write dataframe of tracts, codes, and scores to csv in data_final folder and in flask static folder
df_ages.to_csv('data_final/tracts_scores'+extra+'.csv')
df_ages.to_csv('wheresious/static/tracts_scores'+extra+'.csv')

## Make a histogram and do some stats on the WhereSIOUS scores

In [ ]:
# make histogram of scores scaled 0 to 100
plt.figure(figsize=(13, 10))
plt.hist(df_ages.score, bins=25, histtype='step', align='mid', linewidth=2, \
         label=['Scaled Score From Model'])
plt.legend(loc=2,fontsize=20)
#plt.xlim(0,100)
plt.grid()
plt.tick_params(axis='both', which='major', labelsize=25)
plt.xlabel('WhereSIOUS Score',fontsize=35)
plt.ylabel('# of Census Tracts',fontsize=35)

# print some stats about model output
mean = np.mean(df_ages.score)
print mean
print np.median(df_ages.score)
std = np.std(df_ages.score)
print std

In [ ]:
# get max score
max_score = df_ages.score.max()
print df_ages[max_score==df_ages.score.values]
# get min score
min_score = df_ages.score.min()
print df_ages[min_score==df_ages.score.values]

In [ ]:
# read in scores for all businesses
temp = pd.read_csv('data_final/tracts_scores.csv',index_col=0)

# plot histogram of all scores
plt.figure(figsize=(13, 10))
plt.hist(temp.score, bins=range(0,100,4), histtype='step', align='mid', linewidth=4, \
         label=['Score (All)'],color='k')
plt.text(np.mean(temp.score),38.,'all',fontsize=20)

# same for each specific code we've done
n=0
for extra in extras:
    temp = pd.read_csv('data_final/tracts_scores_'+extra+'.csv',index_col=0)
    plt.hist(temp.score, bins=range(0,100,4), histtype='step', align='mid', linewidth=2, \
         label=['Score ('+extra+')'])
    plt.text(np.mean(temp.score),38*((7-n)/35.+.9),extra,fontsize=20)
    n+=1

plt.legend(loc=0,fontsize=20)
#plt.xlim(0,100)
plt.grid()
plt.tick_params(axis='both', which='major', labelsize=25)
plt.xlabel('WhereSIOUS Score',fontsize=35)
plt.ylabel('# of Census Tracts',fontsize=35)